In [1]:
function batch_processor(f, request::Channel{Tuple{X,Promise}}) where {X,Promise}
    batchsize = 32
    
    # list of images to classify
    # support 32 elements, but if less, don't give empty elements back
    xs = sizehint!(Vector{X}(undef, 0), batchsize)
    promises = sizehint!(Vector{Promise}(undef, 0), batchsize)
    
    # classify the images in batches
    nitems = 0
    try
        for (x, p) in request
            push!(xs, x)
            push!(promises, p)
            nitems += 1
            if nitems == batchsize
                foreach(put!, promises, f(xs))
                empty!(xs)
                empty!(promises)
                nitems = 0
            end
        end
        # classify any leftovers
        if !isempty(xs)
            foreach(put!, promises, f(xs))
        end
        
    catch err
        @error "Error in batch processor calls to BAR" exception = (err, catch_backtrace())
#     catch
#         foreach(close, promises)
#         rethrow()
    end
end

function batched_call(request::Channel{Tuple{X,Channel{Y}}}, x::X) where {X,Y}
    promise = Channel{Y}(1)
    put!(request, (x, promise))
    return take!(promise)
end

function FOO(request::Channel)
    x = 5
    y = batched_call(request, x)
    return x == y
end

function BAR(x)
    return x
end

X = Y = Any
request = Channel{Tuple{X,Channel{Y}}}()   # (32) - maybe better if it's buffered; tuning needed
ntasks = 100

@sync try
    @async try
        batch_processor(BAR, request)
    catch err
        @error "Error in batch processor calls to BAR" exception = (err, catch_backtrace())
    finally
        close(request)
    end
    for _ in 1:ntasks
        Threads.@spawn try
            FOO(request)
        
        catch err
            @error "Error in batch processor calls to BAR" exception = (err, catch_backtrace())
        
#         catch
#             close(request)
#             rethrow()
        end
    end

catch err
    @error "Error in sync calls to FOO" exception = (err, catch_backtrace())
    
finally
    close(request)
end

┌ Error: Error in batch processor calls to BAR
│   exception = (InvalidStateException("Channel is closed.", :closed), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000000168cdb4b6, Ptr{Nothing} @0x0000000168cdb597, Ptr{Nothing} @0x00000001088c6015, Ptr{Nothing} @0x0000000168cda3a8, Ptr{Nothing} @0x0000000168cda66c, Ptr{Nothing} @0x00000001088c6015, Ptr{Nothing} @0x00000001088e125d])
└ @ Main In[1]:69
┌ Error: Error in batch processor calls to BAR
│   exception = (InvalidStateException("Channel is closed.", :closed), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000000110efd466, Ptr{Nothing} @0x0000000110f21ee6, Ptr{Nothing} @0x0000000168cdadde, Ptr{Nothing} @0x0000000168cdb3df, Ptr{Nothing} @0x0000000168cdb597, Ptr{Nothing} @0x00000001088c6015, Ptr{Nothing} @0x0000000168cda3a8, Ptr{Nothing} @0x0000000168cda66c, Ptr{Nothing} @0x00000001088c6015, Ptr{Nothing} @0x00000001088e125d])
└ @ Main In[1]:69
┌ Error: Error in batch processor calls to BAR
│   exception = (Inva